In [1]:
import random
import numpy as np
import pandas as pd

In [2]:
count=50

In [3]:
def beta_unigrams(emotion):
    df=pd.read_csv("/kaggle/input/word-gen/unigram_emotion.csv")
    df=df[emotion]
    df=df.to_numpy()
    row=df
    row=row/sum(row)
#     print(len(row),sum(row),"row_uni")
    return row

def p_first(corpus,uniquewords) :
    with open(corpus) as f:
        lines = f.readlines()
    word={}
    total=0
    for line in lines:
        words=line.split()
        if words[0] in word:
            word[words[0]]+=1
            total+=1
        else:
            word[words[0]]=1
            total+=1
        for j in words:
            if(j not in word):
                word[j]=1
                total+=1
    probs=[]
    # print(len(word.keys()))
    for i in uniquewords:
        if i in word:
            probs.append(word[i]/total)
        else:
            probs.append(0)
    
#     print(total,len(probs),sum(probs),"first_prob")
    # print(probs)
    probs=np.array(probs)
    return probs

types=["No","Laplace","KneserNey"]
emotions=["sadness","joy","love","anger","fear","surprise"]
unigram_emotions=[beta_unigrams(i) for i in emotions]

bigram_sadness=pd.read_csv("/kaggle/input/word-gen/sadness_scores.csv")
bigram_joy=pd.read_csv("/kaggle/input/word-gen/joy_scores.csv")
bigram_love=pd.read_csv("/kaggle/input/word-gen/love_scores.csv")
bigram_anger=pd.read_csv("/kaggle/input/word-gen/anger_scores.csv")
bigram_fear=pd.read_csv("/kaggle/input/word-gen/fear_scores.csv")
bigram_surprise=pd.read_csv("/kaggle/input/word-gen/surprise_scores.csv")

No_smoothing=pd.read_csv("/kaggle/input/word-gen/No_Smoothing.csv")
Laplace_smoothing=pd.read_csv("/kaggle/input/word-gen/Laplace_Smoothing.csv")
KneserNey_smoothing=pd.read_csv("/kaggle/input/word-gen/KneserNey_Smoothing.csv")

uniquewords=KneserNey_smoothing["Unnamed: 0"]
uniquewords=uniquewords.to_numpy()
uniquewords=list(uniquewords)


first_prob=p_first("/kaggle/input/word-gen/corpus.txt",uniquewords)


def generate_sentences(sentence_length,alpha,beta,uniquewords,emotion,type):
        global first_prob,unigram_emotions
        # r=random.choice(self.uniquewords)
        # first_word=random.choise(uniquewords,weights=first_generator(0),k=1)[0]
        beta_unigram_dist=unigram_emotions[emotions.index(emotion)]
        first_prob_dist=first_prob*beta+beta_unigram_dist*(1-beta)
        # print(len(first_prob_dist),first_prob_dist.dtype,sum(first_prob_dist))
        first_word=random.choices(uniquewords,weights=first_prob_dist,k=1)[0]
        s=first_word+" "
        k=uniquewords.index(first_word)
        for i in range(sentence_length-1):
            beta_unigram_dist=unigram_emotions[emotions.index(emotion)]
            beta_bigram_dist=beta_bigrams(k,emotion)
            smoothed_probabs=prob_distribution(k,type)
#             prob_dist=alpha*smoothed_probabs+(1-alpha/2)*beta_unigram_dist+(1-alpha/2)*beta_bigram_dist
            prob_dist=alpha*smoothed_probabs+(1-alpha)*beta_bigram_dist
#             prob_dist=alpha*smoothed_probabs+(1-alpha)*beta_unigram_dist
#             print(sum(prob_dist),"distribution")
            # prob_dist=alpha*smoothed_probabs+(1-alpha)*beta_unigram_dist
            w=random.choices(uniquewords,weights=prob_dist,k=1)[0]
            # w=random.choices(self.uniquewords,weights=self.co_matrix[k],k=1)[0]
            s+=w+" "
            k=uniquewords.index(w)
        return s



def beta_bigrams(k,emotion):
    global bigram_sadness,bigram_joy,bigram_love,bigram_anger,bigram_fear,bigram_surprise
    idx=k
    #row idx of df
    if(emotion=="sadness"):
        df=bigram_sadness
    elif(emotion=="joy"):
        df=bigram_joy
    elif(emotion=="love"):
        df=bigram_love
    elif(emotion=="anger"):
        df=bigram_anger
    elif(emotion=="fear"):
        df=bigram_fear
    elif(emotion=="surprise"):
        df=bigram_surprise
    

    row=df.iloc[idx]
    row=row.to_numpy()
    row=row[1:]
    row=row/sum(row)
    # print(len(row))
    return row

def prob_distribution(k,type):
    global No_smoothing,Laplace_smoothing,KneserNey_smoothing

    if(type=="No"):
        df=No_smoothing
    elif(type=="Laplace"):
        df=Laplace_smoothing
    elif(type=="KneserNey"):
        df=KneserNey_smoothing

    idx=k
    #row idx of df
    row=df.iloc[idx]
    row=row.to_numpy()
    row=row[1:]
    # print(row)
    row=row/sum(row)
    # print(len(row))
    return row

    # print(uniquewords)

5429 0.9999999999999983 row_uni
5429 1.0000000000000022 row_uni
5429 0.999999999999998 row_uni
5429 0.9999999999999992 row_uni
5429 1.0000000000000033 row_uni
5429 0.9999999999999986 row_uni
7823 5429 0.9999999999999606 first_prob


In [4]:
# print(generate_sentences(10,0.5,uniquewords,"joy"))

for j in emotions:
    for k in types:
        sentences=[]
        for i in range(count):
            l=random.randint(8,30)
            sentences.append(generate_sentences(l,0.5,0.9,uniquewords,j,k))    
        all="\n".join(sentences)
        print(j,k)
        with open("Generated_"+j+"_"+k+".txt","w") as f:
            f.write(all)

2.000000000000005 distribution
1.999999999999999 distribution
2.0000000000000018 distribution
1.999999999999989 distribution
1.9999999999999984 distribution
1.9999999999999991 distribution
1.9999999999999931 distribution
2.000000000000001 distribution
2.000000000000005 distribution
1.9999999999999973 distribution
2.0000000000000018 distribution
2.0000000000000013 distribution
1.9999999999999916 distribution
1.999999999999989 distribution
1.9999999999999971 distribution
1.9999999999999951 distribution
1.9999999999999916 distribution
2.0 distribution
2.000000000000006 distribution
1.9999999999999944 distribution
1.9999999999999976 distribution
1.9999999999999916 distribution
1.9999999999999973 distribution
1.9999999999999978 distribution
1.9999999999999982 distribution
sadness No
2.0000000000000457 distribution
2.000000000000054 distribution
2.000000000000052 distribution
2.0000000000000893 distribution
2.000000000000044 distribution
2.000000000000048 distribution
2.0000000000000417 dist